## Загрузка и подготовка данных

In [36]:
# Загружаем библиотеки
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from lightfm import LightFM
import implicit
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.dump import dump
from collections import defaultdict

In [8]:
# Загружаем данные
events = pd.read_csv("data/events.csv.zip")
properties = pd.concat([
    pd.read_csv("data/item_properties_part1.csv.zip"),
    pd.read_csv("data/item_properties_part2.csv.zip")
])

In [9]:
# Преобразуем timestamp в формат datetime
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms')

In [10]:
# Создаём отдельные временные признаки на основе времени события
events['day_of_week'] = events['event_datetime'].map(lambda x: x.weekday())
events['Year'] = events['event_datetime'].map(lambda x: x.year)
events['Month'] = events['event_datetime'].map(lambda x: x.month)
events['Day'] = events['event_datetime'].map(lambda x: x.day)
events['Hour'] = events['event_datetime'].map(lambda x: x.hour)
events['minute'] = events['event_datetime'].map(lambda x: x.minute)

In [11]:
# Создаём функцию, чтобы сгенерировать признаки отрезка дня на основе часа
def get_time_periods(hour):
    if 3 <= hour < 7:
        return 'Dawn'
    elif 7 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

# Применяем функцию
events['Day Period'] = events['Hour'].map(get_time_periods)

# Удаляем лишнее
events.drop(['event_datetime', 'timestamp'], axis=1, inplace=True)
properties.drop(['timestamp'], axis=1, inplace=True)

In [12]:
# Оставляем только пользователей с не менее чем 5 взаимодействиями
user_interactions = events['visitorid'].value_counts()
user_interactions = user_interactions[user_interactions >= 5].index
events = events[events['visitorid'].isin(user_interactions)]

In [13]:
# Выбираем топ-свойства
top_properties = properties.drop_duplicates(['itemid', 'property']).groupby("property")['itemid'].count().sort_values(ascending=False)[:20]
properties_filtered = properties[properties['property'].isin(set(top_properties.index))].reset_index()
properties_filtered.drop('index', axis=1, inplace=True)

In [14]:
# Смотрим на пустые значения, а именно, в каких столбцах они есть и сколько процентов от всего объема они занимают
for column in events.columns[events.isna().any()].tolist():
  empty_percentage = (events[column].isna().sum() / len(events)) * 100
  print(f"Процент пустых значений в столбце '{column}': {empty_percentage:.2f} %")

Процент пустых значений в столбце 'transactionid': 98.07 %


In [15]:
# 98% - это очень много, признак неинформативный, удаляем его
events.drop('transactionid', axis=1, inplace=True)

In [16]:
# Смотрим на структуру датасета
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 948537 entries, 1 to 2756095
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   visitorid    948537 non-null  int64 
 1   event        948537 non-null  object
 2   itemid       948537 non-null  int64 
 3   day_of_week  948537 non-null  int64 
 4   Year         948537 non-null  int64 
 5   Month        948537 non-null  int64 
 6   Day          948537 non-null  int64 
 7   Hour         948537 non-null  int64 
 8   minute       948537 non-null  int64 
 9   Day Period   948537 non-null  object
dtypes: int64(8), object(2)
memory usage: 79.6+ MB


In [17]:
# Кодируем нечисловые признаки
label_encoder = LabelEncoder()
events['event_encoded'] = label_encoder.fit_transform(events['event'])
events['day_period_encoded'] = label_encoder.fit_transform(events['Day Period'])

# Удаляем лишнее
events = events.drop(['event', 'Day Period'], axis=1).reset_index(drop=True)

In [18]:
# Посмотрим на properties_filtered
print(properties_filtered.info())
print('========================')
print(properties_filtered.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13563669 entries, 0 to 13563668
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   itemid    int64 
 1   property  object
 2   value     object
dtypes: int64(1), object(2)
memory usage: 310.4+ MB
None
   itemid    property                    value
0  460429  categoryid                     1338
1  206783         888  1116713 960601 n277.200
2   59481         790               n15360.000
3  156781         917                   828513
4  285026   available                        0


In [19]:
# В value мы имеем странные значения, посмотрим сколько всего уникальных значений в этом столбце
properties_filtered.value.nunique()

1899949

In [20]:
# Почти 2 млн - очень много, поэтому не будем кодировать, а будем обрабатывать

# Создаём функцию, которая уберёт лишние символы и оставит только цифры
# В виде эксперимента, оставляем только ту часть числа, что идёт в начале до первого пробела (если имеется)
def extract_numbers_from_string(value_str):
    numbers = re.findall(r'\d+', value_str)
    cleaned_value = ' '.join(numbers)
    first_number = re.findall(r'\d+', cleaned_value)
    if first_number:
        return first_number[0]
    else:
        return 0

In [21]:
# Применяем функцию extract_numbers_from_string к столбцу 'value'
properties_filtered['cleaned_value'] = properties_filtered['value'].apply(extract_numbers_from_string)
# Преобразуем столбец 'cleaned_value' в числовой формат
properties_filtered['cleaned_value'] = properties_filtered['cleaned_value'].apply(pd.to_numeric, errors='coerce')
# Удаляем лишний столбец
properties_filtered.drop(['value'], axis=1, inplace=True)

In [22]:
# Смотрим на второй категориальный признак в наборе данных properties_filtered - property
properties_filtered.property.nunique()

20

In [23]:
# Всего 20 уникальных значений, поэтому будем их кодировать
properties_features = pd.get_dummies(properties_filtered, columns=['property'], prefix=['prop'], drop_first=True)

In [24]:
# Объединяем данные из events и из properties_features
combined_data = events.merge(properties_features, on='itemid', how='left')

In [25]:
# Смотрим на пустые значения, а именно, в каких столбцах они есть и сколько процентов от всего объема они занимают
for column in combined_data.columns[combined_data.isna().any()].tolist():
  empty_percentage = (combined_data[column].isna().sum() / len(combined_data)) * 100
  print(f"Процент пустых значений в столбце '{column}': {empty_percentage:.2f} %")

Процент пустых значений в столбце 'cleaned_value': 0.08 %
Процент пустых значений в столбце 'prop_159': 0.08 %
Процент пустых значений в столбце 'prop_202': 0.08 %
Процент пустых значений в столбце 'prop_227': 0.08 %
Процент пустых значений в столбце 'prop_28': 0.08 %
Процент пустых значений в столбце 'prop_283': 0.08 %
Процент пустых значений в столбце 'prop_364': 0.08 %
Процент пустых значений в столбце 'prop_6': 0.08 %
Процент пустых значений в столбце 'prop_678': 0.08 %
Процент пустых значений в столбце 'prop_689': 0.08 %
Процент пустых значений в столбце 'prop_698': 0.08 %
Процент пустых значений в столбце 'prop_764': 0.08 %
Процент пустых значений в столбце 'prop_776': 0.08 %
Процент пустых значений в столбце 'prop_790': 0.08 %
Процент пустых значений в столбце 'prop_839': 0.08 %
Процент пустых значений в столбце 'prop_888': 0.08 %
Процент пустых значений в столбце 'prop_917': 0.08 %
Процент пустых значений в столбце 'prop_928': 0.08 %
Процент пустых значений в столбце 'prop_avai

In [26]:
# Проценты ничтожные, поэтому заполняем их медианами
medians = combined_data.median()
combined_data = combined_data.fillna(medians)

In [27]:
# Преобразуем все признаки к int64
combined_data = combined_data.astype('int64')

> Обучать модели будем 3 модели: LightFM, ALS и SVG. При этом обучим их по 2 раза: как на датасете events, так и на объединённых данных combined_data.

> В итоге, у нас будет 6 результов, из которых и будем выбирать лучший.

In [28]:
# Для каждой модели потребуется по 2 набора тестовых и обучающих данных, поэтому создадим их

train_events, test_events = train_test_split(events, test_size=0.3, shuffle=False, random_state=42)
train_combined, test_combined = train_test_split(combined_data, test_size=0.3, shuffle=False, random_state=42)

## LightFM

> Обучаем модель только на events

In [ ]:
# Преобразуем данные в разреженную матрицу в формате COO
train_interactions_events = coo_matrix((np.ones(len(train_events)), (train_events['visitorid'], train_events['itemid'])))
test_interactions_events = coo_matrix((np.ones(len(test_events)), (test_events['visitorid'], test_events['itemid'])))

In [ ]:
# Перед обучением проверяем есть ли пропуски в созданных матрицах
print(np.isnan(train_interactions_events.data).any())
print(np.isnan(test_interactions_events.data).any())

False
False


In [ ]:
# Создаём и обучаем модель LightFM
model_lightfm_events = LightFM(loss='warp', no_components=100, learning_rate=0.1, random_state=42)
model_lightfm_events.fit(train_interactions_events, epochs=100, num_threads=8)

In [ ]:
from lightfm.evaluation import precision_at_k

# Оцениваем модель LightFM с использованием метрики Precision@3
precision_test = precision_at_k(model_lightfm_events, test_interactions_events, k=3).mean()
print("Precision@3 for LightFM (events only): {:.2f}%".format(precision_test * 100))

Precision@3 for LightFM (events only): 5.39%


> Обучаем модель на combined_data

In [ ]:
# Преобразуем данные в разреженную матрицу в формате COO
train_interactions_combined = coo_matrix((np.ones(len(train_combined)), (train_combined['visitorid'], train_combined['itemid'])))
test_interactions_combined = coo_matrix((np.ones(len(test_combined)), (test_combined['visitorid'], test_combined['itemid'])))

In [ ]:
# Перед обучением проверяем есть ли пропуски в созданных матрицах
print(np.isnan(train_interactions_combined.data).any())
print(np.isnan(test_interactions_combined.data).any())

False
False


In [ ]:
# Создаём и обучаем модель LightFM
model_lightfm_combined = LightFM(loss='warp', no_components=100, learning_rate=0.1, random_state=42)
model_lightfm_combined.fit(train_interactions_combined, epochs=100, num_threads=8)

In [ ]:
# Оцениваем модель LightFM с использованием метрики Precision@3
precision_test = precision_at_k(model_lightfm_combined, test_interactions_combined, k=3).mean()
print("Precision@3 for LightFM (combined data): {:.2f}%".format(precision_test * 100))

Precision@3 for LightFM (combined data): 6.48%


## ALS

> Обучаем модель только на events

In [ ]:
# Преобразуем данные в формат CSR
train_csr_events = train_interactions_events.T.tocsr()
test_csr_events = test_interactions_events.T.tocsr()

# Создаем модель ALS
model_als_events = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=100)

# Обучаем модель на разреженной матрице взаимодействий
model_als_events.fit(train_csr_events)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Получаем топ-3 рекомендации для всех пользователей
recommendations = model_als_events.recommend_all(train_csr_events, N=3)

# Преобразуем рекомендации в словарь
top3_recommendations_als = {user_id: recs for user_id, recs in enumerate(recommendations)}

In [ ]:
from implicit.evaluation import precision_at_k

# Считаем Precision@3
precision_at_3_als = precision_at_k(model_als_events, train_csr_events, test_csr_events, K=3)
print("Precision@3 for ALS (events only): {:.2f}%".format(precision_at_3_als * 100))

  0%|          | 0/56030 [00:00<?, ?it/s]

Precision@3 for ALS (events only): 0.61%


> Обучаем модель на combined_data

In [ ]:
# Преобразуем данные в формат CSR
train_csr_combined = train_interactions_combined.T.tocsr()
test_csr_combined = test_interactions_combined.T.tocsr()

# Создаем модель ALS
model_als_combined = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=100)

# Обучаем модель на разреженной матрице взаимодействий
model_als_combined.fit(train_csr_combined)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Получаем топ-3 рекомендации для всех пользователей
recommendations = model_als_combined.recommend_all(train_csr_combined, N=3)

# Преобразуем рекомендации в словарь
top3_recommendations_als = {user_id: recs for user_id, recs in enumerate(recommendations)}

In [ ]:
# Считаем Precision@3
precision_at_3_als = precision_at_k(model_als_combined, train_csr_combined, test_csr_combined, K=3)
print("Precision@3 for ALS (combined data): {:.2f}%".format(precision_at_3_als * 100))

  0%|          | 0/56127 [00:00<?, ?it/s]

Precision@3 for ALS (combined data): 1.34%


## SVD

> Обучаем модель только на events

In [29]:
from surprise.model_selection import train_test_split

# Создаём объект Reader
reader = Reader(rating_scale=(1, 5))

# Создаём объект Dataset из ваших данных
data = Dataset.load_from_df(events[['visitorid', 'itemid', 'event_encoded']], reader)

# Разделяем данные на обучающий и тестовый наборы
trainset_events, testset_events = train_test_split(data, test_size=0.3, random_state=42)

In [33]:
# Обучаем модель SVD
model_svd_events = SVD()
model_svd_events.fit(trainset_events)

# Получаем предсказания для всех пользователей в тестовом наборе данных
test_predictions = model_svd_events.test(testset_events)

In [34]:
# Создаём словарь, чтобы хранить топ-3 рекомендации для каждого пользователя
top3_recommendations = defaultdict(list)

for uid, iid, true_r, est, _ in test_predictions:
    # Проверяем, есть ли пользователь в trainset_events
    if trainset_events.knows_user(uid):
        top3_recommendations[uid].append((iid, est))

# Сортируем рекомендации для каждого пользователя по оценкам и берём топ-3
for uid, user_ratings in top3_recommendations.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top3_recommendations[uid] = user_ratings[:3]

In [35]:
# Вычисляем Precision@3
total_precision = 0
for uid, user_ratings in top3_recommendations.items():
    # Проверяем, есть ли пользователь в trainset_events
    if trainset_events.knows_user(uid):
        true_items = set(test_events[test_events['visitorid'] == uid]['itemid'])
        recommended_items = set([iid for (iid, _) in user_ratings])
        precision = len(true_items & recommended_items) / 3
        total_precision += precision

average_precision = total_precision / len(top3_recommendations)

print("Precision@3 for SVD (events only): {:.2f}%".format(average_precision * 100))

Precision@3 for SVD (events only): 19.56%


> Обучаем модель на combined_data

In [ ]:
# Создаём объект Reader
reader = Reader(rating_scale=(1, 5))

# Создаём объект Dataset из ваших данных
data = Dataset.load_from_df(combined_data[['visitorid', 'itemid', 'event_encoded']], reader)

# Разделяем данные на обучающий и тестовый наборы
trainset_combined, testset_combined = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
# Обучаем модель SVD
model_svd = SVD()
model_svd.fit(trainset_combined)

# Получаем предсказания для всех пользователей в тестовом наборе данных
test_predictions = model_svd.test(testset_combined)

In [ ]:
# Создаём словарь, чтобы хранить топ-3 рекомендации для каждого пользователя
top3_recommendations = defaultdict(list)

for uid, iid, true_r, est, _ in test_predictions:
    # Проверяем, есть ли пользователь в trainset_combined
    if trainset_combined.knows_user(uid):
        top3_recommendations[uid].append((iid, est))

# Сортируем рекомендации для каждого пользователя по оценкам и берём топ-3
for uid, user_ratings in top3_recommendations.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top3_recommendations[uid] = user_ratings[:3]

In [ ]:
# Вычисляем Precision@3
total_precision = 0
for uid, user_ratings in top3_recommendations.items():
    # Проверяем, есть ли пользователь в trainset_combined
    if trainset_combined.knows_user(uid):
        true_items = set(test_combined[test_combined['visitorid'] == uid]['itemid'])
        recommended_items = set([iid for (iid, _) in user_ratings])
        precision = len(true_items & recommended_items) / 3
        total_precision += precision

average_precision = total_precision / len(top3_recommendations)

print("Precision@3 for SVD (combined data): {:.2f}%".format(average_precision * 100))

Precision@3 for SVD (combined data): 10.64%


## Заключение

Исходя из результатов Precision@3 для различных моделей и данных, можно сделать следующие выводы:

> **LightFM**:
На данных только из событий (events only) Precision@3 составляет 5.39%, а на объединенных данных (combined data) - 6.48%. Это может указывать на более успешное обучение модели на более полном объеме данных. Общий результат довольно неплохой для системы рекомендаций.

> **ALS (Alternating Least Squares)**:
Precision@3 для ALS на данных только из событий невелик (0.61%), но при использовании объединенных данных результат немного выше (1.34%). ALS, вероятно, не так хорошо подходит для этих данных, и объединение данных может дать некоторое улучшение, но всё равно остается низким.

> **SVD (Singular Value Decomposition)**:
Precision@3 для SVD на данных только из событий высокий (19.56%), но при использовании объединенных данных снижается до 10.64%. Это может быть связано с различиями в структуре данных и обратной связи между ними. Может быть, SVD лучше работает с более чистыми данными.

Итак, общий вывод может быть следующим: для наших данных модель SVD работает лучше на событиях, а не на объединенных данных. LightFM также показывает приемлемую производительность и может быть предпочтительным выбором в зависимости от контекста использования и целей системы рекомендаций. ALS, кажется, менее эффективен в данном контексте.

In [37]:
# Сохраняем наиболее эффективную модель
dump("model_svd_events.pkl", algo=model_svd_events)

In [ ]:
!pip install lightfm
!pip install implicit
!pip install scikit-surprise